In [2]:
import pandas as pd 
import numpy as np
import re
from datetime import datetime


In [77]:
recipes_df = pd.read_csv('data/clean_recipes.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [79]:
recipes_df.head(2)

,Unnamed: 0,RecipeId,Name,CookTime,PrepTime,TotalTime,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions
0,0,38,Low-Fat Berry Blue Frozen Dessert,PT24H,PT45M,PT24H45M,Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,1,39,Biryani,PT25M,PT4H,PT4H25M,Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...",3.0,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,"c(""Soak saffron in warm milk for 5 minutes and..."


### Columns to Drop 

In [80]:
recipes_df = recipes_df.drop(columns = ['AuthorId',
                                  'AuthorName',
                                  'DatePublished',
                                   'Description',
                                  'Images'])

KeyError: "['AuthorId' 'AuthorName' 'DatePublished' 'Description' 'Images'] not found in axis"

In [ ]:
recipes_df = recipes_df.drop(columns = ['RecipeYield'])

### Drop columns with missing values

In [15]:
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141788 entries, 0 to 141787
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  141788 non-null  int64  
 1   RecipeId                    141788 non-null  int64  
 2   Name                        141788 non-null  object 
 3   CookTime                    141788 non-null  object 
 4   PrepTime                    141788 non-null  object 
 5   TotalTime                   141788 non-null  object 
 6   RecipeCategory              141788 non-null  object 
 7   Keywords                    141788 non-null  object 
 8   RecipeIngredientQuantities  141788 non-null  object 
 9   RecipeIngredientParts       141788 non-null  object 
 10  AggregatedRating            141788 non-null  float64
 11  ReviewCount                 141788 non-null  float64
 12  Calories                    141788 non-null  float64
 13  FatContent    

In [81]:
recipe_clean_df = recipes_df.dropna(how='any',axis=0) 

In [82]:
recipe_clean_df = recipe_clean_df.reset_index()

New dataset is 141,788 rows now with no missing values. This is more than enough to train our model.

In [18]:
#recipe_clean_df.to_csv('data/clean_recipes.csv')

## Prep Time and Cook Time into floats

In [83]:
# remove the string character 
recipe_clean_df['CookTime'] = recipe_clean_df['CookTime'].str.replace(r'PT', '')
recipe_clean_df['PrepTime'] = recipe_clean_df['PrepTime'].str.replace(r'PT', '')
recipe_clean_df['TotalTime'] = recipe_clean_df['TotalTime'].str.replace(r'PT', '')

In [84]:
# Convert to date time
recipe_clean_df['CookTime'] = recipe_clean_df['CookTime'].str.replace(r'M', '')
recipe_clean_df['PrepTime'] = recipe_clean_df['PrepTime'].str.replace(r'M', '')
recipe_clean_df['TotalTime'] = recipe_clean_df['TotalTime'].str.replace(r'M', '')

In [21]:
# # convert the hours in to minutes for Cooktime column
# for count, value in enumerate(recipe_clean_df['CookTime']):
#     if type(recipe_clean_df['CookTime'][count]) == int:
#         break
#     elif 'H' in recipe_clean_df['CookTime'][count]:
#         recipe_clean_df['CookTime'][count] = (int(recipe_clean_df['CookTime'][0][:-1])) * 60

In [22]:
# convert the hours in to minutes for PrepTime column
# for count, value in enumerate(recipe_clean_df['PrepTime']):
#     if 'H' in recipe_clean_df['PrepTime'][count]:
#         recipe_clean_df['PrepTime'][count] = (int(recipe_clean_df['PrepTime'][0][:-1])) * 60

### Function to convert string time to minutes

In [86]:
def time_converter(column):
    for count, value in enumerate(recipe_clean_df[f'{column}']):
        if type(value) == str:
            for count2, item in enumerate(value):
                if item == 'H':
                    if value[-1:]  in 'H':
                        recipe_clean_df[f'{column}'][count] = float(recipe_clean_df[f'{column}'][count][:value.index(item)]) * 60
                    else:
                        recipe_clean_df[f'{column}'][count] = float(recipe_clean_df[f'{column}'][count][:value.index(item)]) * 60 + (float(recipe_clean_df[f'{column}'][count][value.index(item)+1:]))

In [87]:
time_converter('TotalTime')
time_converter('PrepTime')
time_converter('CookTime')

/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [71]:
recipe_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141788 entries, 0 to 141787
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   index                       141788 non-null  int64  
 1   Unnamed: 0                  141788 non-null  int64  
 2   RecipeId                    141788 non-null  int64  
 3   Name                        141788 non-null  object 
 4   CookTime                    141788 non-null  object 
 5   PrepTime                    141788 non-null  object 
 6   TotalTime                   141788 non-null  object 
 7   RecipeCategory              141788 non-null  object 
 8   Keywords                    141788 non-null  object 
 9   RecipeIngredientQuantities  141788 non-null  object 
 10  RecipeIngredientParts       141788 non-null  object 
 11  AggregatedRating            141788 non-null  float64
 12  ReviewCount                 141788 non-null  float64
 13  Calories      

In [30]:
recipe_df_1 = recipe_clean_df.copy()

In [31]:
# recipe_clean_df.to_csv('./data/recipes141.csv', sep='\t', encoding='utf-8')

In [115]:
recipes10k = recipe_clean_df[:10_000]

In [99]:
# recipes10k.to_csv('./data/recipes10.csv', sep='\t', encoding='utf-8')

In [31]:
recipes10_df = pd.read_csv('data/recipes10.csv', error_bad_lines=False, encoding='utf-8')

In [30]:
recipes10k.to_csv('./data/recipes10.csv', sep=',', encoding='utf-8')

NameError: name 'recipes10k' is not defined

### Change string to a list

In [32]:
def list_convert(dataframe, column):
    for count, value in enumerate(dataframe[f'{column}']): 
        dataframe[f'{column}'][count] = dataframe[f'{column}'][count].replace('c(', '')
        dataframe[f'{column}'][count] = dataframe[f'{column}'][count].replace(')', '')
        
        dataframe[f'{column}'][count] = dataframe[f'{column}'][count].split(",")
#         dataframe[f'{column}'][count] = dataframe[f'{column}'][count].spl
#     recipe_clean_df[f'{column}'] = recipe_clean_df[f'{column}'].str.replace('"', '')
    
#     dataframe[f'{column}'] = dataframe[f'{column}'].apply(lambda x: list(dataframe[f'{column}']))
    

In [33]:
list_convert(recipes10_df, 'Keywords')

/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

In [40]:
list_convert(recipes10_df, 'RecipeIngredientQuantities')

/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

In [41]:
list_convert(recipes10_df, 'RecipeIngredientParts')

/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

In [42]:
list_convert(recipes10_df, 'RecipeInstructions')

/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

In [48]:
recipes10_df = recipes10_df.drop(columns = ['Unnamed: 0', 'index', 'Unnamed: 0.1'])

In [49]:
recipes10_df.to_csv('./data/recipes10k_df.csv', sep=',', encoding='utf-8')

#### No cook time over four hours

In [68]:
recipes10_df = recipes10_df[(recipes10_df['CookTime'] <= 240) & (recipes10_df['TotalTime'] <= 480)]

#### Check Data is in correct range 

In [69]:
recipes10_df = recipes10_df[(recipes10_df['RecipeServings'] > 0) & (recipes10_df['RecipeServings'] < 20)]

#### Return recipes where Number of reviews is greater than 2

In [71]:
recipes10_df = recipes10_df[recipes10_df['ReviewCount'] > 2]

#### Create calories per serving column 

In [72]:
recipes10_df['calories_per_serving'] = recipes10_df['Calories']/recipes10_df['RecipeServings']

In [80]:
recipes10_df = recipes10_df[recipes10_df['calories_per_serving'] < 500].reset_index().drop(columns = 'index')

In [81]:
recipes10_df.to_csv('./data/recipes5k_df.csv', sep=',', encoding='utf-8')